In [48]:
import pandas as pd
import geopandas as gpd
import shapely
import polyline
import folium
import urllib
import json

In [49]:
requests =  pd.read_csv('requests.csv')
requests.head()

,Timestamp,Requested_Pickup,Requested_Dropoff,Actual_Pickup,Actual_Dropoff
0,1380949200,1149,725,218,3601
1,1380949200,1225,1714,2003,3601
2,1380949200,1237,3143,1314,2218
3,1380949200,1323,3566,1330,2573
4,1380949200,1328,2466,1323,2473


In [50]:
manhattan_geojson = gpd.read_file('manhattan.geojson')
manhattan_geojson.head()

,id,w,geometry
0,0.0,NaN,POINT (-73.9975944 40.7140611)
1,1.0,NaN,POINT (-73.9980743 40.7137811)
2,2.0,NaN,POINT (-73.9615093 40.7602403)
3,3.0,NaN,POINT (-73.9692158 40.763488)
4,4.0,NaN,POINT (-73.9694382 40.7635824)


In [51]:
manhattan = manhattan_geojson.reset_index()[['index', 'geometry']]
manhattan_requests = manhattan.merge(requests, left_on = 'index', right_on ='Requested_Pickup')
manhattan_requests = manhattan_requests.merge(manhattan, left_on = 'Requested_Dropoff', right_on ='index')
manhattan_requests = manhattan_requests.rename(columns={"geometry_x": "Requested_Pickup_geo", "geometry_y": "Requested_Dropoff_geo"})
manhattan_requests = manhattan_requests[['Timestamp', 'Requested_Pickup_geo', 'Requested_Dropoff_geo', 'Actual_Pickup', 'Actual_Dropoff']]
manhattan_requests = manhattan_requests.merge(manhattan, left_on = 'Actual_Pickup', right_on ='index')
manhattan_requests = manhattan_requests.merge(manhattan, left_on = 'Actual_Dropoff', right_on ='index')
manhattan_requests = manhattan_requests.rename(columns={"geometry_x": "Actual_Pickup_geo", "geometry_y": "Actual_Dropoff_geo"})
manhattan_requests = manhattan_requests[['Timestamp', 'Requested_Pickup_geo', 'Requested_Dropoff_geo', 'Actual_Pickup_geo', 'Actual_Dropoff_geo']]
manhattan_requests.head()

,Timestamp,Requested_Pickup_geo,Requested_Dropoff_geo,Actual_Pickup_geo,Actual_Dropoff_geo
0,1380949320,POINT (-73.9975944 40.7140611),POINT (-73.98730070000001 40.7446783),POINT (-73.996861 40.7150654),POINT (-73.9895958 40.7415497)
1,1381030251,POINT (-73.99349359999999 40.7216185),POINT (-73.98730070000001 40.7446783),POINT (-73.9913214 40.7194205),POINT (-73.9895958 40.7415497)
2,1380985369,POINT (-74.01354929999999 40.715284),POINT (-73.98730070000001 40.7446783),POINT (-74.01354929999999 40.715284),POINT (-73.9895958 40.7415497)
3,1380987120,POINT (-73.98639300000001 40.745921),POINT (-73.9879839 40.7408836),POINT (-73.9884802 40.7468087),POINT (-73.9895958 40.7415497)
4,1380979006,POINT (-73.99027599999999 40.756567),POINT (-73.98730070000001 40.7446783),POINT (-73.987447 40.755373),POINT (-73.9895958 40.7415497)


In [52]:
def request_location(manhattan_requests, i):
    requested_pk_latlon = str(manhattan_requests['Requested_Pickup_geo'][i].y) +','+ str(manhattan_requests['Requested_Pickup_geo'][i].x)
    requested_dp_latlon = str(manhattan_requests['Requested_Dropoff_geo'][i].y) +','+ str(manhattan_requests['Requested_Dropoff_geo'][i].x)
    
    endpoint = 'https://maps.googleapis.com/maps/api/directions/json?'
    api_key = '**********'
    
    nav_request = 'origin={}&destination={}&key={}'.format(requested_pk_latlon, requested_dp_latlon ,api_key)
    request = endpoint + nav_request
    #Sends the request and reads the response.
    response = urllib.request.urlopen(request).read()
    #Loads response as JSON
    directions = json.loads(response)
    
    return directions

In [53]:
distance = request_location(manhattan_requests, 1)['routes'][0]['legs'][0]['distance']['text']
distance

'2.2 mi'

In [54]:
duration = request_location(manhattan_requests, 1)['routes'][0]['legs'][0]['duration']['text']
duration

'16 mins'

In [55]:
line = polyline.decode(request_location(manhattan_requests, 1)['routes'][0]['overview_polyline']['points'])

In [56]:
fmap = folium.Map(location=[40.7140611, -73.9975944], zoom_start=13)
folium.RegularPolygonMarker(location=[line[0][0], line[0][1]], radius=3, color='red').add_to(fmap)
folium.RegularPolygonMarker(location=[line[-1][0], line[-1][1]], radius=3, color='red').add_to(fmap)
for i in range(len(line)-1):
    folium.PolyLine(locations=[line[i], line[i+1]]).add_to(fmap)
    
fmap